# Energy renovation project for a building

#### Package import

In [1]:
import random
import numpy as np
import pandas as pd

#### Inputs
Here you can change the values for investment costs, number of building elements and the m number, which is defined for the normalized problem as the maximum count of measures over all building elements.

In [2]:
# The total renovation investment costs
Inv = 1000
# the total number of building elements
n = 10
# the maximum count of measures over all building elements for a normalized problem 
m = 5

#### Parameters of the measures 
Values related to material costs and heat loss costs after 20 years of usage are set randomly for experimental purposes. Their values can also be loaded from other data sources, such as CSV-files.

##### Experimental runs
- Material costs are taken with 1 decimal point and generated using samples from a uniform distribution from low value of 50 to the highest value of 300 price unit
- Heat loss costs after 20 years of usage from 10 to 1000 units

In [3]:
d = {"e": [], "Kij": [], "Hij": []}
for i in range(n):
    d["e"].append(i)
    # Material costs
    d["Kij"].append(np.round(np.random.uniform(low=50, high=300, size=(m-1,)), decimals=1))
    # Heat loss costs TODO("Rethink low and high values")
    d["Hij"].append(np.round(np.random.uniform(low=10, high=1000, size=(m,)), decimals=1))

#### Step 1:

As mentioned in the Condition 2 in README.md file, the renovation of a building element can be left out, limitation either due to no sufficient investment amount or because the impact of leaving the building element without any renovation is irrelevant.

So, we should add 0.0 price unit to the dictionary d and consequently to the dataframe df.

In [4]:
for i in range(len(d["Kij"])):
    d["Kij"][i] = np.insert(d["Kij"][i], [0], [0], axis=0)
df = pd.DataFrame(data=d)

##### Diplay the dataframe df

In [5]:
df.head()

,e,Kij,Hij
0,0,"[0.0, 276.6, 239.7, 180.6, 109.5]","[56.5, 418.3, 300.0, 477.1, 104.4]"
1,1,"[0.0, 74.5, 102.1, 257.7, 156.3]","[134.8, 898.0, 625.5, 193.9, 788.1]"
2,2,"[0.0, 134.2, 80.6, 91.4, 59.8]","[694.0, 17.7, 152.0, 171.9, 794.9]"
3,3,"[0.0, 124.8, 246.2, 190.2, 104.9]","[651.6, 414.8, 131.2, 65.4, 245.9]"
4,4,"[0.0, 211.0, 123.3, 202.9, 59.3]","[493.2, 423.9, 251.9, 494.0, 408.6]"


#### Step 2: towards the fractional knapsack problem

Find the indices in the dataframe of the greedy renovation measures. 

To do so, we calculate $\min_{e}$ for each e building element. We define hereafter some helpers variables and find sort their values.

In [6]:
indices = []
helper = []
Min = []
for i in df.index:
    product = [km * kg for (km, kg) in zip(df["Kij"][i], df["Hij"][i])]
    indices.append((i, product.index(min([value for value in product if value > 0]))))
    helper.append(i)
for i in df.index:
    k = helper.index(i)
    Min.append((i, np.round(df["Kij"][i][indices[k][1]] * df["Hij"][i][indices[k][1]], decimals=1)))

#### Step 3: sort & reorder

In [7]:
# sort ascendingly
Min.sort(key=lambda x: x[1])
dt = pd.DataFrame(data=Min)
dt.columns = ["e", "Min_e"]

##### Diplay the dataframe dt

Note that the elements are well ascendingly sorted.

In [8]:
dt.head()

,e,Min_e
0,2,2375.3
1,7,9764.2
2,5,10936.4
3,0,11431.8
4,3,12439.1


#### Step 4:
Invest Inv starting with the lowest value of $\min_{e}$ with some help-variables

In [9]:
# Display for the project's inputs:
print("Renovation project's infos:")
print("\tThe number of building elements:\t{}.".format(n))
print("\tThe total renovation investment costs:\t{} price unit.".format(Inv))
# Display for the investement plan:
print("\nInvestement plan with ordered measures:")
count = 0
toInvest = Inv
sum = 0
counter = 0
while toInvest > 0 and count < len(Min):
    flag = helper.index(Min[count][0])
    if toInvest > df["Kij"][Min[count][0]][indices[flag][1]]:
        toInvest -= df["Kij"][Min[count][0]][indices[flag][1]]
        print(
            "\tRenovation measure {}\tKij = {} price unit.\t\tHij = {} price unit.".format(Min[count][0], df["Kij"][Min[count][0]][indices[flag][1]],
                                     df["Hij"][Min[count][0]][indices[flag][1]]))
        sum += df["Kij"][Min[count][0]][indices[flag][1]]
        counter += 1
    else:
        pass
    count += 1
print("\n\tNumber of chosen buiding elements for renovation: {}.".format(counter))
print("\tInvested sum: {} price unit.".format(sum))

Renovation project's infos:
	The number of building elements:	10.
	The total renovation investment costs:	1000 price unit.

Investement plan with ordered measures:
	Renovation measure 2	Kij = 134.2 price unit.		Hij = 17.7 price unit.
	Renovation measure 7	Kij = 121.9 price unit.		Hij = 80.1 price unit.
	Renovation measure 5	Kij = 149.2 price unit.		Hij = 73.3 price unit.
	Renovation measure 0	Kij = 109.5 price unit.		Hij = 104.4 price unit.
	Renovation measure 3	Kij = 190.2 price unit.		Hij = 65.4 price unit.
	Renovation measure 4	Kij = 59.3 price unit.		Hij = 408.6 price unit.
	Renovation measure 8	Kij = 209.1 price unit.		Hij = 144.1 price unit.

	Number of chosen buiding elements for renovation: 7.
	Invested sum: 973.4 price unit.
